In [2]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

In [10]:
prompt.template

'Answer the user query.\n{format_instructions}\n{query}\n'

In [4]:
prompt.pretty_print()

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```
{query}



---

NotImplementedError: Unsupported message type: <class 'dict'>

In [13]:
type(format_instructions)

str

In [15]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

# Define your desired data structure
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

# Set up a parser and inject instructions into the ChatPromptTemplate
parser = PydanticOutputParser(pydantic_object=Joke)
format_instructions = parser.get_format_instructions()

chat_prompt = ChatPromptTemplate.from_messages([
    {"role": "system", "content": "You are a helpful AI assistant that tells jokes."},
    {"role": "user", "content": "Answer the user query.\n{format_instructions}\n{query}\n"},
])

# Use the ChatPromptTemplate with the parser
model = ChatOpenAI(temperature=0.0)
chat_prompt_and_model = chat_prompt | model
output = chat_prompt_and_model.invoke({"query": "Tell me a joke."})

# Parse the output using the PydanticOutputParser
parsed_output = parser.invoke(output)
print(parsed_output)

NotImplementedError: Unsupported message type: <class 'dict'>

In [18]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

# Define your desired data structure
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

# Set up a parser and inject instructions into the ChatPromptTemplate
parser = PydanticOutputParser(pydantic_object=Joke)
format_instructions = parser.get_format_instructions()

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant that tells jokes."),
    ("human", "Answer the user query.\n{format_instructions}\n{query}\n"),
])

# Use the ChatPromptTemplate with the parser
model = ChatOpenAI(temperature=0.0)
chat_prompt_and_model = chat_prompt | model
output = chat_prompt_and_model.invoke({"query": "Tell me a joke.",
                                       "format_instructions": format_instructions})

# Parse the output using the PydanticOutputParser
parsed_output = parser.invoke(output)
print(parsed_output)

setup="Why couldn't the bicycle find its way home?" punchline='Because it lost its bearings!'
